In [1]:
pip install surya-ocr

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


In [3]:
from PIL import Image
from surya.recognition import RecognitionPredictor
from surya.detection import DetectionPredictor

/Users/nevidujayatilleke/Documents/MSC - Research/OCR Comparative Analysis/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
image = Image.open("example_text_ransaka.jpg")
langs = ["ta"] # Replace with your languages or pass None (recommended to use None)

In [5]:
recognition_predictor = RecognitionPredictor()

Loaded recognition model s3://text_recognition/2025_02_18 on device mps with dtype torch.float16


In [6]:
detection_predictor = DetectionPredictor()

Loaded detection model s3://text_detection/2025_02_28 on device mps with dtype torch.float16


In [7]:
predictions = recognition_predictor([image], [langs], detection_predictor)

Recognizing Text: 100%|██████████| 1/1 [00:02<00:00,  2.03s/it]


In [8]:
predictions

[OCRResult(text_lines=[TextLine(polygon=[[0.0, 17.0], [2424.0, 9.0], [2543.0, 98.0], [13.0, 107.0]], confidence=0.9833984375, text='ඔහු සතුව තිබී හෙරොයින් කිලෝග්\u200dරෑම් 1ක් සොයා ගත් අතර ඔහු  ', bbox=[0.0, 9.0, 2543.0, 107.0])], languages=['ta'], image_bbox=[0.0, 0.0, 2600.0, 128.0])]

In [9]:
import pandas as pd
from datasets import load_dataset

In [10]:
dataset = load_dataset("Nevidu/tamil_synthetic_ocr")

In [11]:
print(dataset)

DatasetDict({
    data: Dataset({
        features: ['image', 'text'],
        num_rows: 7000
    })
})


In [12]:
train_dataset = dataset["data"]

In [13]:
def perform_ocr(image, langs = ["si"]):
    try:
        predictions = recognition_predictor([image], [langs], detection_predictor)
        # predictions = detection_predictor([image])
        for result in predictions:
          for line in result.text_lines:
              prediction = line.text
        return prediction.strip()
    except Exception as e:
        print(f"Error during OCR: {e}")
        return None

In [14]:
from tqdm import tqdm

In [15]:
results = []
for i in tqdm(range(len(dataset['data']))):
    item = dataset['data'][i]
    image = item['image']
    reference_text = item['text']
    generated_text = perform_ocr(image)
    results.append({'reference': reference_text, 'generated': generated_text})

100%|██████████| 7000/7000 [2:34:57<00:00,  1.33s/it]


In [16]:
results

[{'reference': 'இங்கச ர்ந்தமழப கப்படம எடத்தப்பதமக்கயம் இல்லய',
  'generated': 'இங்கச ரந்தமழப கப்படம் எடத்தப்பதமக்கயம் இல்லய'},
 {'reference': 'ஆண்ட உனக்கபதத இல்ல ஆனல்நீ இம்மறகற்றஞ்சட்டப்பட்டவரன்பக்கம்',
  'generated': 'ஆண்ட உனக்கபதத இல்ல ஆனல்நீ இம்மறகற்றஞ்சட்டப்பட்டவரன்பக்கம்'},
 {'reference': 'பர்களம்தயர் எல்லரம்தய ர் கபத்தல்கந்தளக்கம்',
  'generated': 'பர்களமதயர் எல்லர்மதய ரக்பதத்தலகந்தளக்கம்'},
 {'reference': 'அவர் எங்களடன்பணயற்றத்தடங்கம்பதஅவர்கள்சய்வர்கள்',
  'generated': 'அவர் எங்களடன்பணயற்றத்தடங்கம்பதஅவர்களச்யவர்கள'},
 {'reference': 'நான் நாளை ஒரு சிகிச்சை வேண்டும் நீங்கள் அதை செய்ய முடியும்',
  'generated': 'நான் நாளை ஒரு சிகிச்சை வேண்டும் நீங்கள் அதை செய்ய முடியும்'},
 {'reference': 'நன்உங்கள்தய்க்கஅளத்த வக்கறத யநறவற்றமயற்ச க்கறன்',
  'generated': 'நன்உங்களதயக்கஅளத்த வக்கறத் யநறவற்றம்யற்ச ககறன'},
 {'reference': 'மன்னள் மறயஎஸ்ச னட்உற ப்பனர் மற்றம்பனப்பர்பரள',
  'generated': '.'},
 {'reference': 'தலஆறவக்கறதக்ககசரயனப்ப ய் பர்ப்பமஞபகம்இரக்க',
  'generated': 'தலஆறவக்கறதக்கசரயனப் ய ப

In [17]:
df = pd.DataFrame(results)

In [18]:
df.head(20)

,reference,generated
0,இங்கச ர்ந்தமழப கப்படம எடத்தப்பதமக்கயம் இல்லய,இங்கச ரந்தமழப கப்படம் எடத்தப்பதமக்கயம் இல்லய
1,ஆண்ட உனக்கபதத இல்ல ஆனல்நீ இம்மறகற்றஞ்சட்டப்பட்...,ஆண்ட உனக்கபதத இல்ல ஆனல்நீ இம்மறகற்றஞ்சட்டப்பட்...
2,பர்களம்தயர் எல்லரம்தய ர் கபத்தல்கந்தளக்கம்,பர்களமதயர் எல்லர்மதய ரக்பதத்தலகந்தளக்கம்
3,அவர் எங்களடன்பணயற்றத்தடங்கம்பதஅவர்கள்சய்வர்கள்,அவர் எங்களடன்பணயற்றத்தடங்கம்பதஅவர்களச்யவர்கள
4,நான் நாளை ஒரு சிகிச்சை வேண்டும் நீங்கள் அதை செ...,நான் நாளை ஒரு சிகிச்சை வேண்டும் நீங்கள் அதை செ...
5,நன்உங்கள்தய்க்கஅளத்த வக்கறத யநறவற்றமயற்ச க்கறன்,நன்உங்களதயக்கஅளத்த வக்கறத் யநறவற்றம்யற்ச ககறன
6,மன்னள் மறயஎஸ்ச னட்உற ப்பனர் மற்றம்பனப்பர்பரள,.
7,தலஆறவக்கறதக்ககசரயனப்ப ய் பர்ப்பமஞபகம்இரக்க,தலஆறவக்கறதக்கசரயனப் ய பரப்பமஞ்பகமஇர்க்க
8,கறந்தபட்சம்நன்மடந்ததஎன்றசந்த ஷமகஇர க்கம் ஒரவரன...,கறந்தபட்சம்நன்மடந்ததஎன்றசந்த ஷமகஇர் க்கம் ஒரவர...
9,நீங்கள்தன்அந்தமர ந்த எழதக்கண்டரப்பீர்கள்எனநனக்...,நிங்களதனஅந்தமர ந்த எழுதககண்டரப்பர்கள்என்றனக்கறன


In [19]:
df.to_csv("Surya_OCR_Results.csv", index = False)

In [20]:
from jiwer import wer, cer

In [21]:
agg_error_wer = 0
agg_error_cer = 0
for i in range(len(df)):
    record = df.iloc[i]
    try:
        error_wer = wer(record['reference'], record['generated'])
        error_cer = cer(record['reference'], record['generated'])
    except:
        print(record)
        print(i)
    agg_error_wer += error_wer
    agg_error_cer += error_cer

In [22]:
print(f"Word Error Rate (WER): {agg_error_wer/ len(df)}")

Word Error Rate (WER): 0.6563400603364914


In [23]:
print(f"Character Error Rate (CER): {agg_error_cer/ len(df)}")

Character Error Rate (CER): 0.1547909735989841


In [24]:
results

[{'reference': 'இங்கச ர்ந்தமழப கப்படம எடத்தப்பதமக்கயம் இல்லய',
  'generated': 'இங்கச ரந்தமழப கப்படம் எடத்தப்பதமக்கயம் இல்லய'},
 {'reference': 'ஆண்ட உனக்கபதத இல்ல ஆனல்நீ இம்மறகற்றஞ்சட்டப்பட்டவரன்பக்கம்',
  'generated': 'ஆண்ட உனக்கபதத இல்ல ஆனல்நீ இம்மறகற்றஞ்சட்டப்பட்டவரன்பக்கம்'},
 {'reference': 'பர்களம்தயர் எல்லரம்தய ர் கபத்தல்கந்தளக்கம்',
  'generated': 'பர்களமதயர் எல்லர்மதய ரக்பதத்தலகந்தளக்கம்'},
 {'reference': 'அவர் எங்களடன்பணயற்றத்தடங்கம்பதஅவர்கள்சய்வர்கள்',
  'generated': 'அவர் எங்களடன்பணயற்றத்தடங்கம்பதஅவர்களச்யவர்கள'},
 {'reference': 'நான் நாளை ஒரு சிகிச்சை வேண்டும் நீங்கள் அதை செய்ய முடியும்',
  'generated': 'நான் நாளை ஒரு சிகிச்சை வேண்டும் நீங்கள் அதை செய்ய முடியும்'},
 {'reference': 'நன்உங்கள்தய்க்கஅளத்த வக்கறத யநறவற்றமயற்ச க்கறன்',
  'generated': 'நன்உங்களதயக்கஅளத்த வக்கறத் யநறவற்றம்யற்ச ககறன'},
 {'reference': 'மன்னள் மறயஎஸ்ச னட்உற ப்பனர் மற்றம்பனப்பர்பரள',
  'generated': '.'},
 {'reference': 'தலஆறவக்கறதக்ககசரயனப்ப ய் பர்ப்பமஞபகம்இரக்க',
  'generated': 'தலஆறவக்கறதக்கசரயனப் ய ப

In [25]:
len(results)

7000